In [ ]:
import pandas as pd
from math import pi
import numpy as np
import matplotlib.pyplot as plt
from pypdf import PdfMerger
from jinja2 import Template
import pdfkit
import base64

**Требуемые файлы:**
- subindex.xlsx
- gg.txt
- page_html_template_2.txt

### Считаем рейтинг равенства по регионам.

In [5]:
data = pd.read_excel('indexes_for_region.xlsx')
print(data.shape)
data.head()

(84, 20)


,Регион,Уровень участия в составе рабочей силы населения (%),Средняя начисленная заработная плата за октябрь (руб.),Доля на руководящих должностях за октябрь (%),Численность среди специалистов высшего уровня квалификации,Экономическое участие и возможности Субиндекс,Численность с основным общим образованием (9 классов) в рабочей силе,Численность со средним общим образованием (11 классов) в рабочей силе,Численность с высшим образованием в рабочей силе,Уровень образования Субиндекс,"Ожидаемая продолжительность жизни при рождении, число лет",Число родившихся (чел.),Здоровье и выживание Субиндекс,Доля на гос. должностях в федеральных государственных органах (%),Доля на гос. должностях в органах судебной власти и прокуратуры (%),Доля на муниципальных должностях и должностях муниципальной службы (%),Политические возможности Субиндекс,Индекс гендерного неравенства,"Численность населения, тыс.",Доля женщин (%)
0,Белгородская область,0.785319,0.787608,1.000000,1.0,0.922743,1.0,1.0,1.0,0.982154,1,1.000000,1.000000,1.0,1.0,1.0,1.0,0.976224,1531.9,53.678439
1,Брянская область,0.762186,0.757782,0.984127,1.0,0.911678,1.0,1.0,1.0,1.000000,1,0.913007,0.976188,1.0,1.0,1.0,1.0,0.971966,1168.8,54.175222
2,Владимирская область,0.775887,0.772457,1.000000,1.0,0.918591,1.0,1.0,1.0,1.000000,1,1.000000,1.000000,1.0,1.0,1.0,1.0,0.979648,1323.7,54.453426
3,Воронежская область,0.784978,0.787878,0.869159,1.0,0.907554,1.0,1.0,1.0,1.000000,1,0.943515,0.984538,1.0,1.0,1.0,1.0,0.973023,2287.7,54.067404
4,Ивановская область,0.779494,0.845056,1.000000,1.0,0.928881,1.0,1.0,1.0,1.000000,1,0.917601,0.977445,1.0,1.0,1.0,1.0,0.976582,976.9,54.560344


In [6]:
data.columns

Index(['Регион', 'Уровень участия в составе рабочей силы населения (%)',
       'Средняя начисленная заработная плата за октябрь (руб.)',
       'Доля на руководящих должностях за октябрь (%)',
       'Численность среди специалистов высшего уровня квалификации',
       'Экономическое участие и возможности Субиндекс',
       'Численность с основным общим образованием (9 классов) в рабочей силе',
       'Численность со средним общим образованием (11 классов) в рабочей силе',
       'Численность с высшим образованием в рабочей силе',
       'Уровень образования Субиндекс',
       'Ожидаемая продолжительность жизни при рождении, число лет',
       'Число родившихся (чел.)', 'Здоровье и выживание Субиндекс',
       'Доля на гос. должностях в федеральных государственных органах (%)',
       'Доля на гос. должностях в органах судебной власти и прокуратуры (%)',
       'Доля на муниципальных должностях и должностях муниципальной службы (%)',
       'Политические возможности Субиндекс', 'Индекс

Переименуем колонки для html

In [7]:
html_data = pd.DataFrame()

html_data['reg'] = data['Регион']
html_data['res'] = round(data['Индекс гендерного неравенства'], 3)
html_data['num'] = data['Численность населения, тыс.']
html_data['num_fm'] = round(data['Доля женщин (%)'], 3)

# экономическая сфера
html_data['sub_1'] = round(data['Экономическое участие и возможности Субиндекс'], 3)
html_data['sub_11'] = round(data['Уровень участия в составе рабочей силы населения (%)'], 3)
html_data['sub_12'] = round(data['Средняя начисленная заработная плата за октябрь (руб.)'], 3)
html_data['sub_13'] = round(data['Доля на руководящих должностях за октябрь (%)'], 3)
html_data['sub_14'] = round(data['Численность среди специалистов высшего уровня квалификации'], 3)

# сфера образования
html_data['sub_2'] = round(data['Уровень образования Субиндекс'], 3)
html_data['sub_21'] = round(data['Численность с основным общим образованием (9 классов) в рабочей силе'], 3)
html_data['sub_22'] = round(data['Численность со средним общим образованием (11 классов) в рабочей силе'], 3)
html_data['sub_23'] = round(data['Численность с высшим образованием в рабочей силе'], 3)

# сфера здравоохранения
html_data['sub_3'] = round(data['Здоровье и выживание Субиндекс'], 3)
html_data['sub_31'] = round(data['Ожидаемая продолжительность жизни при рождении, число лет'], 3)
html_data['sub_32'] = round(data['Число родившихся (чел.)'], 3)

# политическая сфера
html_data['sub_4'] = round(data['Политические возможности Субиндекс'], 3)
html_data['sub_41'] = round(data['Доля на гос. должностях в федеральных государственных органах (%)'], 3)
html_data['sub_42'] = round(data['Доля на гос. должностях в органах судебной власти и прокуратуры (%)'], 3)
html_data['sub_43'] = round(data['Доля на муниципальных должностях и должностях муниципальной службы (%)'], 3)

print(html_data.shape)
html_data.head(4)

(84, 20)


,reg,res,num,num_fm,sub_1,sub_11,sub_12,sub_13,sub_14,sub_2,sub_21,sub_22,sub_23,sub_3,sub_31,sub_32,sub_4,sub_41,sub_42,sub_43
0,Белгородская область,0.976,1531.9,53.678,0.923,0.785,0.788,1.000,1.0,0.982,1.0,1.0,1.0,1.000,1,1.000,1.0,1.0,1.0,1.0
1,Брянская область,0.972,1168.8,54.175,0.912,0.762,0.758,0.984,1.0,1.000,1.0,1.0,1.0,0.976,1,0.913,1.0,1.0,1.0,1.0
2,Владимирская область,0.980,1323.7,54.453,0.919,0.776,0.772,1.000,1.0,1.000,1.0,1.0,1.0,1.000,1,1.000,1.0,1.0,1.0,1.0
3,Воронежская область,0.973,2287.7,54.067,0.908,0.785,0.788,0.869,1.0,1.000,1.0,1.0,1.0,0.985,1,0.944,1.0,1.0,1.0,1.0


In [8]:
def calc_ranks(numbers):
    '''
    вычисляем ранги элементов в списке чисел в порядке убывания
    '''
    sorted_numbers = sorted(numbers, reverse=True)
    rank_dict = {}
    for rank, num in enumerate(sorted_numbers):
        if num not in rank_dict:
            rank_dict[num] = rank + 1
    ranks = [rank_dict[num] for num in numbers]
    return ranks

example_nums = [11, 14, 10, 15, 14]
print(calc_ranks(example_nums))

[4, 2, 5, 1, 2]


In [9]:
html_data.columns

Index(['reg', 'res', 'num', 'num_fm', 'sub_1', 'sub_11', 'sub_12', 'sub_13',
       'sub_14', 'sub_2', 'sub_21', 'sub_22', 'sub_23', 'sub_3', 'sub_31',
       'sub_32', 'sub_4', 'sub_41', 'sub_42', 'sub_43'],
      dtype='object')

In [10]:
rank_columns = ['res', 'sub_1', 'sub_11', 'sub_12', 'sub_13', 'sub_14', 'sub_2', 'sub_21', 'sub_22', 'sub_23', \
                'sub_3', 'sub_31', 'sub_32', 'sub_4', 'sub_41', 'sub_42', 'sub_43']

for column in rank_columns:
  html_data[column + '_r'] = calc_ranks(html_data[column])

print(html_data.shape)
html_data.head(4)

(84, 37)


,reg,res,num,num_fm,sub_1,sub_11,sub_12,sub_13,sub_14,sub_2,...,sub_21_r,sub_22_r,sub_23_r,sub_3_r,sub_31_r,sub_32_r,sub_4_r,sub_41_r,sub_42_r,sub_43_r
0,Белгородская область,0.976,1531.9,53.678,0.923,0.785,0.788,1.000,1.0,0.982,...,1,1,1,1,1,1,1,1,1,1
1,Брянская область,0.972,1168.8,54.175,0.912,0.762,0.758,0.984,1.0,1.000,...,1,1,1,78,1,78,1,1,1,1
2,Владимирская область,0.980,1323.7,54.453,0.919,0.776,0.772,1.000,1.0,1.000,...,1,1,1,1,1,1,1,1,1,1
3,Воронежская область,0.973,2287.7,54.067,0.908,0.785,0.788,0.869,1.0,1.000,...,1,1,1,36,1,36,1,1,1,1


## Создаём пдф-страницу для каждого региона

In [ ]:
def plot_polar_chart(r, save_path="rose.png"):
    
    # задаём массив направлений
    angles = np.arange(0, 2 * pi, pi / 2.0)

    categories = ['Экономика', '', 'Образование', '', 'Здравоохранение', '', 'Политика']

    fig = plt.figure()      # создаём новое полотно, на котором будем рисовать
    ax = fig.add_subplot(111, projection='polar')       # ax - полотно в полярной проекции, занимающее всё пространство полотно fig

    ax.set_theta_direction(-1)      # увеличение угла theta будет по часовой
    ax.set_theta_offset(pi / 2.0)       # изменим положение нуля (ставим его на положение 'N', которое сейчас равно 90 градусам)

    ax.set_xticklabels(categories)
    ax.set_rlabel_position(0)

    # добавляем линию, которая соединяет последнюю и первую точки
    ax.plot((angles[-1], angles[0]), (r[-1], r[0]), color='b', linewidth=2.)

    # рисуем график и его заполнение
    ax.plot(angles, r, color='b', linewidth=2.)
    ax.fill(angles, r, color='blue', alpha=0.1)

    # сохраняем график
    plt.savefig(save_path)
    plt.close(fig) # закрываем фигуру, чтобы освободить память

In [18]:
merger = PdfMerger()

<ipython-input-18-0abb4a5166c5>:1: DeprecationWarning: PdfMerger is deprecated and will be removed in pypdf 5.0.0. Use PdfWriter instead.
  merger = PdfMerger()


In [19]:
df_subind = pd.read_excel('subindex.xlsx')
df_subind.head()

full_df = pd.DataFrame()
full_df['Регион'] = df_subind['Регион']
full_df['Экономика'] = df_subind["Экономическое участие и возможности Субиндекс"]
full_df['Образование'] = df_subind["Уровень образования Субиндекс"]
full_df['Здоровье'] = df_subind['Здоровье и выживание Субиндекс']
full_df['Политика'] = df_subind['Политические возможности Субиндекс']
full_df['Общий рейтинг'] = df_subind["Индекс гендерного неравенства"]

full_df

,Регион,Экономика,Образование,Здоровье,Политика,Общий рейтинг
0,Белгородская область,0.922743,0.982154,1.000000,1.0,0.976224
1,Брянская область,0.911678,1.000000,0.976188,1.0,0.971966
2,Владимирская область,0.918591,1.000000,1.000000,1.0,0.979648
3,Воронежская область,0.907554,1.000000,0.984538,1.0,0.973023
4,Ивановская область,0.928881,1.000000,0.977445,1.0,0.976582
...,...,...,...,...,...,...
79,Амурская область,0.905611,0.987844,0.971628,1.0,0.966271
80,Магаданская область,0.926362,0.966834,1.000000,1.0,0.973299
81,Сахалинская область,0.930611,0.982788,1.000000,1.0,0.978350
82,Еврейская автономная область,0.913440,0.947660,0.975707,1.0,0.959202


In [ ]:
gg_range = [i for i in range(len(full_df)//2)]

rate_columns = ['Общий рейтинг', 'Экономика', 'Образование', 'Здоровье', 'Политика']

for k in range(5):
    column = rate_columns[k]
    full_df = full_df.sort_values(by=column, ascending=False)
    full_df['Рейтинг'] = full_df[column].round(3)
    full_df['Ранг'] = np.arange(full_df.shape[0]) + 1
    full_df.index = np.arange(full_df.shape[0])

    html = open(f'template{k + 1}.txt').read()
    template = Template(html)
    rendered = template.render(gg_range=gg_range, df_gg=full_df)

    with open(f"table{k + 1}.html", "w") as fh:
        fh.write(rendered)

    pdfkit.from_file(f"table{k + 1}.html", f"table{k + 1}.pdf")
    merger.append(f"table{k + 1}.pdf")


In [ ]:
def image_file_path_to_base64_string(filepath: str) -> str:
    with open(filepath, 'rb') as f:
        return base64.b64encode(f.read()).decode()

html = open('page_html_template_2.txt').read()
template = Template(html)


for index, row in html_data.iterrows():

  plot_polar_chart([row['sub_1'], row['sub_2'], row['sub_3'], row['sub_4']], save_path="rose.png")

  params = {
      'reg': row['reg'],
      'res_r': row['res_r'],
      'res': row['res'],
      'num': row['num'],
      'num_fm': row['num_fm'],
      'img_string': image_file_path_to_base64_string('rose.png'),

      'sub_1_r': row['sub_1_r'],
      'sub_1': row['sub_1'],

      'sub_11_r': row['sub_11_r'],
      'sub_11': row['sub_11'],
      'sub_12_r': row['sub_12_r'],
      'sub_12': row['sub_12'],
      'sub_13_r': row['sub_13_r'],
      'sub_13': row['sub_13'],
      'sub_14_r': row['sub_14_r'],
      'sub_14': row['sub_14'],

      'sub_2_r': row['sub_2_r'],
      'sub_2': row['sub_2'],

      'sub_21_r': row['sub_21_r'],
      'sub_21': row['sub_21'],
      'sub_22_r': row['sub_22_r'],
      'sub_22': row['sub_22'],
      'sub_23_r': row['sub_23_r'],
      'sub_23': row['sub_23'],

      'sub_3_r': row['sub_3_r'],
      'sub_3': row['sub_3'],

      'sub_31_r': row['sub_31_r'],
      'sub_31': row['sub_31'],
      'sub_32_r': row['sub_32_r'],
      'sub_32': row['sub_32'],

      'sub_4_r': row['sub_4_r'],
      'sub_4': row['sub_4'],

      'sub_41_r': row['sub_41_r'],
      'sub_41': row['sub_41'],
      'sub_42_r': row['sub_42_r'],
      'sub_42': row['sub_42'],
      'sub_43_r': row['sub_43_r'],
      'sub_43': row['sub_43'],
  }

  rendered = template.render(**params)

  report_filename = f"report_{row['reg'].replace(' ', '_')}.html"
  with open(report_filename, "w") as fh:
    fh.write(rendered)

  pdfkit.from_file(report_filename, report_filename.replace('html', 'pdf'))
  merger.append(report_filename.replace('html', 'pdf'))



<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(categories)
<ipython-input-11-bf81201a81a1>:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(cate

In [ ]:
merger.write("result.pdf")
merger.close()